NAMA : A. MAKMUN ALJI

NIM  : 210411100241

# Konsep Crawling


# Web Crawling dengan Scrapy

Web crawling adalah proses yang penting dalam pengambilan data dari internet. Baik itu untuk mengumpulkan informasi, mengindeks situs web, atau melakukan penelitian, web crawling adalah alat yang kuat. Salah satu pustaka Python yang paling populer untuk melakukan web crawling adalah Scrapy. Dalam tutorial ini, kita akan menjelaskan secara rinci cara menggunakan Scrapy untuk mengumpulkan data dari situs web dengan efisien.

# Apa itu Scrapy?

Scrapy adalah kerangka kerja open source yang dirancang khusus untuk web crawling dan ekstraksi data dari situs web.

Dengan Scrapy, Anda dapat membuat spider, yaitu program Python yang memindai situs web, mengikuti tautan, dan mengambil data sesuai kebutuhan.

Scrapy memberikan alat yang kuat untuk mengelola permintaan HTTP, mengekstrak data dari HTML, XML, atau JSON, dan menyimpan data ke berbagai format, seperti CSV, JSON, atau basis data.

# Persiapan Awal

Sebelum kita mulai dengan Scrapy, pastikan pustaka ini telah terinstal. Pustaka ini dapat menginstalnya menggunakan pip

In [ ]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


# Membuat Spider

Spider adalah komponen utama dalam Scrapy yang melakukan tugas scraping. Sebelum memulai scraping, penting untuk menentukan cara mengekstrak data dari situs web. XPath atau CSS Selector dapat digunakan untuk mengidentifikasi elemen-elemen yang ingin diambil.

Setelah menentukan cara mengekstrak data, saatnya untuk menulis kode spider. Di dalam spider, perlu didefinisikan URL awal, cara mengikuti tautan, dan bagaimana mengekstrak data. Berikut contoh codenya disini saya scarpy data brita online dari liputan6.com dengan ketagori yang saya ambil adalah pemilu disini ouput yang kita hasilkan dari crawling brita tersebut meliputi judul, isi, tanggal & kategori.

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import logging

class ScrapingWeb(scrapy.Spider):
    name = "kompas"
    start_urls = ['https://indeks.kompas.com/?site=bola']

    custom_settings = {
        'FEED_FORMAT': 'csv',
        'FEED_URI': 'bola.csv',
        'LOG_LEVEL': logging.INFO
    }

    article_count = 0
    max_articles = 50

    def parse(self, response):
        articles = response.css('a.article-link::attr(href)').getall()
        logging.info(f"Found {len(articles)} articles.")

        for link in articles:
            if self.article_count >= self.max_articles:
                logging.info("Max articles limit reached.")
                return  # Hentikan parsing jika batas artikel sudah tercapai

            logging.info(f"Scraping article: {link}")
            yield scrapy.Request(link, callback=self.parse_article)

        next_page = response.css('a.paging__link--next::attr(href)').get()
        if next_page and self.article_count < self.max_articles:
            logging.info(f"Found next page: {next_page}")
            yield scrapy.Request(next_page, callback=self.parse)
        else:
            logging.info("No next page found.")

    def parse_article(self, response):
        if self.article_count >= self.max_articles:
            return  # Jika jumlah artikel sudah tercapai, hentikan

        judul = response.css('h1.read__title::text').get()
        waktu = response.css('div.read__time::text').get()
        konten = ''.join(response.xpath('//div[@class="read__content"]//text()[not(ancestor::script)]').getall()).strip()
        kategori = response.css('li.breadcrumb__item:nth-child(2) span::text').get()

        logging.info(f"Parsed article: {judul}, {waktu}, {kategori}")

        if judul and konten and kategori and waktu:
            self.article_count += 1
            yield {
                'judul': judul,
                'waktu': waktu,
                'url': response.url,
                'konten': konten,
                'kategori': kategori,
            }
        else:
            logging.warning("One or more fields are missing data.")

# Memulai proses crawling
process = CrawlerProcess()
process.crawl(ScrapingWeb)
process.start()


2024-09-23 11:20:14 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2024-09-23 11:20:14 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.10.3, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 1.1.1v  1 Aug 2023), cryptography 39.0.1, Platform Windows-10-10.0.22621-SP0
2024-09-23 11:20:14 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20}
2024-09-23 11:20:14 [py.warnings] WARNING: C:\Users\SLM\anaconda3\Lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of S

2024-09-23 11:20:15 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/22/16042268/persib-vs-persija-pertaruhan-harga-diri-macan-kemayoran
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/22/15561808/persib-vs-persija-rizky-ridho-tak-pusingkan-internal-maung-fokus-menang
2024-09-23 11:20:16 [root] INFO: Found next page: https://indeks.kompas.com/?site=bola&page=3
2024-09-23 11:20:16 [root] INFO: Parsed article: Simone Inzaghi Temukan Keanehan Saat Inter Dibekuk AC Milan,  - 23/09/2024, 05:49 WIB, Bola
2024-09-23 11:20:16 [root] INFO: Parsed article: Nippon Fun Golf 2024, Perkuat Hubungan lewat Turnamen Golf ,  - 23/09/2024, 01:15 WIB, Bola
2024-09-23 11:20:16 [root] INFO: Parsed article: Gelandang Berdarah Indonesia MVP Derbi Milan, Reijnders bak Profesor,  - 23/09/2024, 09:00 WIB, Bola
2024-09-23 11:20:16 [root] INFO: Parsed article: Kualifikasi Piala Asia U20 2025: Timnas Indonesia Tanpa Welber Jardim,  - 23/09/2024, 09:30 WIB, 

2024-09-23 11:20:16 [root] INFO: Parsed article: Hasil Crystal Palace Vs Man United, Setan Merah Tertahan,  - 22/09/2024, 01:27 WIB, Bola
2024-09-23 11:20:16 [root] INFO: Found 15 articles.
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/21/22552528/hasil-liverpool-vs-bournemouth-3-0-diaz-dan-kelleher-brilian-pasukan-slot-menang
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/21/22485798/yance-cedera-malut-united-kehilangan-konsentrasi
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/21/22251258/hasil-venezia-vs-genoa-jay-idzes-rasakan-kemenangan-perdana-di-serie-a
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/21/21153278/hasil-malut-united-vs-bali-united-yance-dibawa-ambulans-bali-menang
2024-09-23 11:20:16 [root] INFO: Scraping article: https://bola.kompas.com/read/2024/09/21/20274128/west-ham-vs-chelsea-0-3-enzo-maresca-ikut

# Menyimpan Hasil

Setelah scraping selesai, hasilnya mungkin perlu disimpan. Data dapat menyimpan data ke berbagai format, seperti CSV, JSON, atau basis data. Scrapy memiliki dukungan bawaan untuk beberapa format ini. dari hasil scrapy di atas saya simpan ke dalam folder saya dengan nama foldernya PPW.

In [ ]:
import pandas as pd

2024-09-23 11:20:17 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


In [ ]:
data_bola = pd.read_csv('bola.csv')
data_health = pd.read_csv('health.csv')

kompas = pd.concat([data_bola, data_health])
kompas.to_csv('kompas.csv', index=False)

In [ ]:
kompas

,judul,waktu,url,konten,kategori
0,MotoGP San Marino 2024: Ketika Marquez Menang ...,"- 09/09/2024, 05:00 WIB",https://www.kompas.com/motogp/read/2024/09/09/...,KOMPAS.com - Marc Marquez sukses memenangi Mot...,Bola
1,"Klasemen PON XXI 2024: Futsal Gagal Emas, Jati...","- 09/09/2024, 07:59 WIB",https://www.kompas.com/sports/read/2024/09/09/...,KOMPAS.com - Tim futsal Jawa Timur (Jatim) gag...,Bola
2,Hasil One Pride MMA 82: Frans Sormin Bekuk Jak...,"- 09/09/2024, 14:31 WIB",https://www.kompas.com/sports/read/2024/09/09/...,KOMPAS.com - Frans Lincol Sormin mengalahkan J...,Bola
3,"Indonesia Vs Australia, Kata Manajer Timnas so...","- 09/09/2024, 10:29 WIB",https://bola.kompas.com/read/2024/09/09/102929...,KOMPAS.com - Timnas Indonesia belum menentukan...,Bola
4,Dampak Kehadiran Marteen Paes untuk Persaingan...,"- 09/09/2024, 11:30 WIB",https://bola.kompas.com/read/2024/09/09/113000...,KOMPAS.com - Penampilan penjaga gawang timnas ...,Bola
...,...,...,...,...,...
45,Penurunan Apa Saja yang Terjadi pada Lansia? I...,"- 05/09/2024, 18:00 WIB",https://health.kompas.com/read/24I05180000468/...,"KOMPAS.com - Seiring bertambahnya umur, setiap...",Health
46,Apakah Minum Kopi Pahit Bagus untuk Kesehatan?...,"- 05/09/2024, 16:00 WIB",https://health.kompas.com/read/24I05160000768/...,KOMPAS.com - Kopi kerap diminum untuk mengatas...,Health
47,"Sering Digunakan untuk Diet, Ini Manfaat Keseh...","- 05/09/2024, 13:37 WIB",https://health.kompas.com/read/24I05133717468/...,"KOMPAS.com - Dalam beberapa tahun terakhir, cu...",Health
48,"Kenali ""BEACH"", Akronim untuk Tanda Peringatan...","- 05/09/2024, 14:00 WIB",https://health.kompas.com/read/24I05140000068/...,KOMPAS.com - Gejala kanker ovarium sering kali...,Health
